<img src="out/generated_images.gif" style="float:right;height:200px">
<img src="../../assets/AE.png" style="float:right;height:200px">

# Variational Auto Encoder

This notbook tests the **VAE** using the actual observations from a gym env.

This model is one part of the **WorldModel** included in the *WorlRewardModel* which is responisble to encode the observation <br> for the **MD-RNN model** so it can predict the next observation. 